## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,54.48,71,100,2.89,overcast clouds
1,1,Hobart,AU,-42.8794,147.3294,48.25,96,75,12.66,fog
2,2,Burns Lake,CA,54.2331,-125.7533,49.42,58,100,8.46,overcast clouds
3,3,Khatanga,RU,71.9667,102.5000,34.29,81,85,14.18,overcast clouds
4,4,Shibetsu,JP,43.6590,145.1320,54.30,66,75,11.50,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
22,Puerto Leguizamo,CO,70.45,overcast clouds,-0.1934,-74.7819,
27,Vila Velha,BR,71.55,scattered clouds,-20.3297,-40.2925,
62,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,
91,Albany,US,70.21,scattered clouds,42.6001,-73.9662,
95,Belmonte,BR,72.82,clear sky,-15.8631,-38.8828,
96,Fukue,JP,71.96,broken clouds,32.6881,128.8419,
101,Touros,BR,74.30,overcast clouds,-5.1989,-35.4608,
104,Carbondale,US,74.05,clear sky,37.7273,-89.2168,
105,Najran,SA,74.05,broken clouds,17.4924,44.1277,
107,Kutum,SD,73.89,few clouds,14.2000,24.6667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("",float("NaN"), inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
22,Puerto Leguizamo,CO,70.45,overcast clouds,-0.1934,-74.7819,la casona de juancho
27,Vila Velha,BR,71.55,scattered clouds,-20.3297,-40.2925,Hotel Vitória Palace
62,Puerto Ayora,EC,72.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
95,Belmonte,BR,72.82,clear sky,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
96,Fukue,JP,71.96,broken clouds,32.6881,128.8419,Goto Guesthouse Utojuku & SA Car Rental
101,Touros,BR,74.30,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
104,Carbondale,US,74.05,clear sky,37.7273,-89.2168,Best Western Saluki Inn
105,Najran,SA,74.05,broken clouds,17.4924,44.1277,محمد هادي ال فطيح
107,Kutum,SD,73.89,few clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
116,Hervey Bay,AU,71.02,clear sky,-25.2986,152.8535,Shelly Bay Resort


In [29]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))